In [1]:
# https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/lstm_seq2seq.ipynb

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

In [7]:
# open file
df = pd.read_csv("../data/casual_data_windows.csv")

In [ ]:
# Prepare data
